# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [1]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [2]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [5]:
browser = webdriver.Chrome(executable_path='/Users/haowen/Desktop/python_project/2nd-PyCrawlerMarathon/Homework/chromedriver')
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1


# 以下是用Selenium模擬下拉網頁動作，讓網頁更新

last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
     # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [6]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [10]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
     
    # 連到外部連結，擷取詳細新聞內容
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 國發會：同仁連假曾赴11景點須回報　可申請在家上班

▲國發會主委陳美伶。（圖／記者邱倢芯攝）
記者邱倢芯／台北報導
為了因應新冠肺炎（COVID-19）疫情的防範，國發會主委陳美伶今（6）日表示，在昨日已通知同仁，清明連假期間若是曾赴國家級警報提及的11個景點需回報，並要定時量測體溫做好自主健康管理，若是發生不舒服的情況可申請在家上班。
至於在彈性上班機制上，陳美伶指出，國發會本來就有彈性上班的機制，建議同仁可以多加善用；另外在異地辦公的部分，先前都已經規劃好，且3週前已做過演練。
據了解，國家級警報提及的11個景點包含了，阿里山森林遊樂區、花蓮東大門夜市、嘉義文化路、台南關子嶺、虎頭埤、烏山頭水庫及湖境度假會館等埤塘風景區、高雄興達港、旗山老街、雲林北港朝天宮以及屏東縣南州鄉以南（含墾丁觀光景點）。
----------------------------------------------------------------------

[1] 不斷更新／企業超前部署　清明連假去過11大景點需「居家辦公」總整理

▲新冠肺炎疫情升溫，企業要求員工連假期間去過熱門景點要居家辦公或加強管理。（圖／記者林敬旻攝）
記者余弦妙／台北報導
清明連假期間台灣不少熱門景點皆出現大量人潮，中央指揮中心為防止出現防疫破口也針對11處景點發出國家級警報，並要求民眾在這短期間若是有到過這11個景點的也必須自主管理14天。不過，台灣各大企業似乎也跟上政府「超前部署」腳步，紛紛通知員工，連假期間去過熱門景點就要從4月6日起開始進行「居家辦公」。
以下為要求員工「居家辦公」企業整理：
11：27更新
11：17更新
▼新冠肺炎疫情升溫，企業要求員工連假期間去過熱門景點要居家辦公或加強管理。（圖／達志影像）
----------------------------------------------------------------------

[2] 食品龍頭統一下令：連假去過11景點者居家辦公14天

▲統一超商以民眾一日三餐需求為主打，推出「振興優惠券」搶攻宅經濟。（圖／統一超商提供）
記者林淑慧／台北報導
清明連假風景區人潮引

▲業者說，台灣不只是口罩產量大，而且品質在國際上也被大大稱讚，「我們未來不只供應國內所需，而且可以大量外銷。」（圖／東森新聞）
記者陳俊宏／綜合報導
台灣6日起口罩日產達到1500萬片，這已是日產的最大量，每年口罩產值能夠飆破50億片，產值達到全球第二名；也因此9日開始，口罩新制解除單雙號限制外，大人14天可買9片、小孩14天可買10片。而口罩原料熔噴不織布已漲到原先價格8到10倍，全球一堆國家都在搶。
4天清明連假期間，工作人員沒休息，輪班上工守在機台前，整理口罩分袋裝箱。台灣口罩產量在新冠肺炎疫情爆發前，一天產值大約188萬片，但疫情爆發後短短1個月，暴增產量1300萬片。
之後產線上軌道，經濟部宣布，連假產值增加，6日起可望衝到日產1500萬片，這也是產值的最頂，合計每年口罩產值能夠飆破50億片，產值達到全球第二名。
▲▼4天清明連假期間，工作人員沒休息，輪班上工守在機台前，趕工製造口罩。（圖／東森新聞）
口罩工廠業者鄭永柱接受《東森新聞》訪問指出，不只是口罩產量大，而且品質在國際上也被大大稱讚，「我們未來不只供應國內所需，而且可以大量外銷。」
本月初還會再進三台機器，口罩產值勢必陸續提升，至於日產能有辦法達到總統蔡英文宣示的2000萬片嗎？經濟部表示，還要再協調廠商。
不過，口罩原料熔噴不織布已漲到原先價格8到10倍，全球一堆國家都在搶；再來，機台關鍵組件供不應求。鄭永柱透露，唯一比較困難的是過濾層，國內製造業者應該也有做很多打算。
由於口罩產值又提升，也因此9日新制已解除單雙號限制，大人改成14天可買9片，小孩14天可買10片。至於政府口罩外交需求量則會另開生產線，與指揮中心徵調的口罩量錯開，中央強調，口罩絕對夠用。
▼口罩原料熔噴不織布已漲到原先價格8到10倍，全球一堆國家都在搶。（圖／東森新聞）
----------------------------------------------------------------------

[15] 文曄祥碩換股／談敵意合購併與合意股權交換博奕賽局的見樹與見林

▲文曄董事長鄭文宗（左2）與祥碩董事長沈振來（右2）、祥碩總經理林哲偉（右1）。（圖／資料照，記者周康玉攝）
●盧陽正／ 銘傳大學財務金融學系資深教授暨財務金融研究中心執行長
世界最大IC通路商大聯大(3702)於2019年11月23日至2020

▲網銀堅持維護台灣玩家的發話權益。（圖／翻攝自《劍俠情緣參免費版》） 
實習記者賴韻如／綜合報導
為表達對新冠肺炎疫情犧牲者的深切哀悼，大陸4月4日舉行全國性哀悼活動，許多由大陸發起的遊戲也紛紛公告「臨時維修」。由台灣廠商「網銀國際」所代理的網遊《劍俠情緣叁》中，因為有台灣玩家提及「武漢肺炎」等字句而遭禁言，然而，網銀為了捍衛言論自由，決定與原廠終止合約。
網銀國際代理的《劍俠情緣參免費版》於4日發布「臨時停機維護」公告，指出伺服器將於4日上午九時起進行停機維護，期間玩家將無法上線。未料當天遊戲恢復後，有台灣玩家在公開頻道上留言「好的，中國武漢肺炎」，該帳號隨即被原廠以「說出敏感字詞」遭封鎖，而公開頻道也出現台灣伺服器全體被禁言的情況。
▲西山居遊戲於4日在微博發文表示「嚴肅處理」。（圖／翻攝自微博） 
遊戲原廠西山居遊戲於4日在微博發文寫道，「舉國為抗擊新冠肺炎鬥爭中的犧牲烈士和逝世同胞哀悼之時，我們發現由台灣網銀國際代理的《劍網3》台服聊天頻道中，有用戶惡意發表有關疫情的不當言論」，更表示，「在大義面前，底線不容觸碰！西山居對此必將嚴肅處理！」。
▲《劍俠情緣參免費版》將在2020年05月05日中午12時停止運營。（圖／翻攝自《劍俠情緣參免費版》） 
對此，網銀國際於今（5）日凌晨發布聲明，表示堅持維護台灣玩家的發話權益，「原廠如果堅持要規範不合台灣既有的言論環境，我們將不惜終止合約，停止與中國廠商的任何商務合作。」然而，不久後雙方也再度各自發布公告，指出將於2020年05月05日中午12時停止運營。
----------------------------------------------------------------------

[24] 任立中：大數據行銷只靠分析還不夠

文．洪寶山
不論在任何產業當中，建立自家品牌是各企業一直在追尋的目標，如今新世代數位科技的發展，企業要如何與消費者達到有效的溝通，進而成為新經濟消費市場的領頭羊，是一門大學問。如今，透過國立臺灣大學管理學院全球品牌與行銷研究中心任立中主任的輔導之下，許多企業都能將危機化為轉機。
在目前疫情尚未穩定情況下，許多人紛紛聯想到過去SARS的疫情狀況，但撇除企業經營之外，疫情是否嚴重的影響自身行銷力？任立中淡然表示，其實並不盡然！依照他過去經驗，反倒是九二一大地震時的天災影響較大，當時

▲富士達保險經紀人董事長廖學茂。（圖／富士達保經提供）
記者楊絡懸／台北報導
隨著新冠肺炎（COVID-19）持續升溫，各大公司行號紛紛採取因應措施，希望降低風險，保護員工與客戶。富士達保險經紀人表示，公司在新春後也陸續施行多項行政措施，確保同仁在工作上健康無虞，同時保障所有保戶權益不受影響。
富士達保經指出，目前制定「4大具體防疫措施」，於不過度干擾的前題下，協助所有內外勤同仁減少健康風險，力求所有同仁保護好自己，為保戶提供更好的服務之外，也不讓保戶擔憂。
1、大型活動改以少人多場形式舉辦
新春開工後，富士達保經便決議取消年度春酒暨榮譽會活動，並延期年中前所有大型頒獎典禮，直至疫情趨緩再擇期舉行，避免群聚感染風險。
此外，富士達保經所有教育訓練相關活動也都改採「少人多場」形式辦理，每場次參與人數掌控在30人左右，減少人群密集傳染風險，並同步著手規劃更多線上教育課程，幫助同仁更方便充實專業知識。
2、出國旅遊獎勵改發放獎勵金
於政府公告第三級旅遊警示前，富士達保經就取消年度競賽旅遊獎勵「西班牙與泰國之旅」，且將全額團費改為獎勵金形式，依獎項分別發放3萬元至16萬元不等金額予獲獎同仁；同時嚴格要求所有同仁若須出國得事先報備告知，回國後也需自主居家健康管理，自我管理結束前不得進辦公室，全面落實防疫管控機制，以保障所有同仁的健康。
▲多數員工選擇遠端工作，不接觸人群。（圖／pakutaso，示意圖）
3、全面推動AI機器人服務
富士達保經表示，2019年上線的「富士達AI機器人」服務更完善與便利。透過數位助理，同仁能適時遠距作業，減少通勤出入辦公室頻率，大幅降低風險，更可幫助不便前往辦公室的同仁，了解最新公司公告及商品資訊，即時掌握保單進度，提供客戶最專業的保險服務，保障所有保戶的權益。
4、落實工作防疫準則
富士達保經近期擬定「業務同仁工作防疫準則」要求同仁確實執行，保障自身與保戶的健康安全。首先，同仁們需每日測量體溫，有病狀立即就醫休息。若需拜訪客戶也必須配戴口罩、隨身攜帶乾洗手／酒精，並盡量選擇開放式場地會面，更要避免一同用餐。
另外，富士達保經也呼籲同仁交談需保持1公尺以上距離，所有使用文具也要確實消毒，且不提供重複使用的紙本文宣品，可改採電子檔文件進行說明；每次約訪盡可能於30分鐘內完成，減少業務同仁與保戶雙方風險。
面對疫情，富士達保經董事長廖學茂表示，在外奔

▲雲端發票、電子發票、載具、歸戶讓不少民眾霧煞煞。（圖／記者許力方攝）
記者黃君瀚／綜合報導
財政部上月（3）宣布從109年1-2月期開始，統一發票增開1組六獎，雲端發票則多10萬組500元。近日就有影片提到，電子發票、雲端發票名字實在太類似，很多民眾根本搞不清楚，載具、歸戶等名詞更是看得霧煞煞，就這麼和2億9千500萬元的加碼金擦身而過。
第二項好處是「自動對獎、自動匯款」，全自動完成不用耗眼力，中了獎金直接進戶頭，還免扣千分之四的印花稅。最後一點是「環保」，過去政府每年得印80億張發票，傳統長條版是紙張能回收，現在新款則是感熱紙，無法回收，只能丟一般垃圾進焚化爐處理。
▲傳統長條版（右）的發票越來越少見，多數商家已更換成新的電子發票證明聯，不過這種感熱紙不可回收。
影片中解析了4個關於電子（雲端）發票的專有名詞：
▼影片介紹雲端發票4個專有名詞，同時教學如何申請、綁定使用。
----------------------------------------------------------------------

[41] 亂世投資術！定期定額抗波動　長期投資表現可期

▲ 定期定額投資不必擔憂市場波動。（圖／pixabay）
記者李瑞瑾／綜合報導
市場震盪不斷影響投資信心，在經濟前景尚未明朗之際，投資專家多建議採「定期定額」進場布局，既可分散風險也不用研究進場時點，且長期投資下多半有不錯的投資表現，普遍受到國內投資人認同。為鼓勵定期定額投資，基金網路銷售平台基富通也推出全平台「股票型基金終身定期定額1折」超低申購手續費，讓投資人再省投資成本。
基富通董事長林修銘表示，目前全球經濟局勢仍有諸多不確定，投資人心態也趨於保守，但觀察過去市場也多次經歷震盪，在不確定因素逐一消除後，市場往往都能收復跌幅。以台股來說，近期也同樣受到國際股市影響出現不小的震盪，但本次國際疫情下台灣防疫表現佳，一旦疫情受到控制，經濟有望出現反彈，目前國際疫情後續發展尚不明朗，若採取定期定額方式長線布局台股基金，投資表現值得期待。
林修銘進一步指出，根據投信投顧公會資料顯示，截至2020年2月，國內投信公司定期定額扣款人數總計超過45萬人，每月扣款金額36.2億元，皆較前一年度增加，扣款人數更連續37個月維持在40萬人以上，凸顯定期定額的投資方式深獲國內投資人認同。採用定期定的方式投資，不用

▲中信Line Pay卡結合速食業者麥當勞，推出新回饋方案，搶賺外帶商機。（示意圖／資料照）
記者紀佳妘／台北報導
中信銀行攜手台灣麥當勞，卡友至麥當勞餐廳門市、得來速及歡樂送服務均享中信卡優惠，即日起至今（2020）年6月30日止，每周三至麥當勞餐廳門市刷中信卡消費，單筆滿99元即送蛋捲冰淇淋兌換券，刷LINE Pay卡再享LINE POINTS 3%回饋，搶賺外帶商機。
中信銀行表示，為推動無現金交易，這次與麥當勞再次推出「中信日」活動，刷中信LINE Pay卡再享LINE POINTS 3%回饋；每周一於麥當勞歡樂送網路訂餐刷中信LINE Pay卡，單筆滿250元贈LINE POINTS 50點回饋（每人每周限贈一次）。
感應支付、行動載具刷卡逐漸普及，付款「嗶了就走」可為消費者節省時間，中信銀行與麥當勞餐廳提供線上、線下消費族群皆可享中信卡優惠。根據中信銀行統計，去年於麥當勞餐廳持中信卡消費的刷卡金額年增率達6成，合作推出的「周三冰氛日」，也送出逾15萬支蛋捲冰淇淋，顯見客戶對於「周三冰氛日」的喜愛。
新冠肺炎（COVID-19）疫情影響消費者出門消費的意願，速食店將消費型態改為「稍作停留」的外帶、「拿了就走」的得來速或無接觸式的外送，應可滿足消費者在防疫期間的用餐需求。
中信銀行提醒，防疫期間卡友可於麥當勞餐廳店面刷卡支付，也可利用得來速與歡樂送服務，減少人員接觸，結帳快速又便利。
----------------------------------------------------------------------

[53] 營益率季增年增　時下首選

急跌大幅修正後哪些價值股閃閃發亮？去年第四季季報透露玄機，產業能見度佳、公司競爭力明顯轉好者，為優先選擇。
文．馮泉富
在FED推出無上限QE以緩解金融市場流動性之後，美股終於迎來大漲，道瓊指數創下歷史漲點紀錄，單日飆漲二一一二點，慣性改變，反彈波啟動。不過，投資人仍舊忐忑不安，因為接下來得面對新冠病毒擴散對實體經濟的重創結果。
高失業率受疫情影響  美股以利空出盡反應
市場的擔憂無可厚非，首先，IHS Markit新公布的三月歐元區綜合採購經理人(PMI)初值僅三一．四，不但創歷史新低，還改寫史上單月最大跌點。歐元區服務業PMI更是從二月的五二．六崩跌到三月的二八．四，創下一九九八年開始此項統計以

▲新冠肺炎疫情襲擊金融市場，渣打銀預估，美元在第2季表現仍偏強，但下半年可能見頂回弱。（示意圖／路透社）
記者紀佳妘／台北報導
隨著新冠肺炎（COVlD-19）疫情來勢洶洶，各國央行也掀起降息風潮，同時政府也祭出財政刺激措施提振經濟，投資人關注匯市布局方向，渣打銀行預期，美元在第2季表現仍偏強，但下半年可能見頂回弱，而人民幣、歐元、英鎊則得視疫情發展狀況而定，至於日圓兌美元可望落在105至112區間波動。
渣打銀行財富管理總處負責人陳太齡表示，3月下旬各國央行在一周內合計降息39碼，力道前所未見，包括美國聯準會（Fed）也緊急降息至零利率，更祭出無限量化寬鬆，但近期市場風險情緒尚未消退，預期在第2季美元仍將偏強，不過由於持續大規模QE，美元走勢可能在下半年見頂回弱。
歐洲與英國同樣因疫情蔓延，重創市場信心，陳太齡指出，歐洲央行祭出負利率和擴大資產購買計劃的效果仍有待觀察，不過在美元相對偏強的前提下，預期第2季歐元兌美元可能處於1.1區間震盪的走勢，英鎊兌美元則可能落在1.15至1.25區間震盪，若相關防疫措施奏效，使得疫情獲得控制，則有機會重新站回1.3。
至於亞洲貨幣方面，疫情衝擊觀光，加上東奧確定延期，勢必衝擊日本經濟，連帶不利日圓表現，但避險需求與不利因素相互拉鋸，陳太齡預期，日圓兌美元可能落在105至112區間波動。
陳太齡認為，由於中國大陸疫情較早爆發，相對比較早受到控制，人民幣走勢相對於其它貨幣持穩，短線可能在7至7.2偏弱格局，但如果疫情獲得控制，有機會在第2季底回到6.86。
此外，全球經濟動盪不安，持續推升黃金避險需求，各國央行降息與資產購買計畫，也支撐金價表現，加上供給面短缺，渣打銀行預期，金價在第2季有機會挑戰1,750美元。
渣打銀行提醒，投資人面對市場的短期波動與不確定性，應保持冷靜，適時降低槓桿和風險資產的集中度，維持去風險化並分散投資的多元配置策略，關注投資的長期性和多樣性。
----------------------------------------------------------------------

[62] 不再當月光族　無痛存錢四步驟

文．蔡武穆
上次我們提到，理財重要的不是投資，而是存一筆隨時可動用的「緊急預備金」，許多人認為「我的薪水就固定這麼多，怎麼存錢？」其實，低薪不是無法存錢的藉口，自制力和執行力才是存

新冠肺炎（COVID-19）重創全球經濟，各國央行也紛紛推出量化寬鬆或降息來救市，而台灣央行使用的政策工具包括降息；降低法定存款準備率；公開市場操作，例如買進債券，釋出貨幣；選擇性信用管制以及道德勸說。此外，許多企業近期也紛紛釋出財報，在《雲端最有錢》節目中，達人們也從財報和籌碼雙優股當中，建議投資人可以留意哪些績優股。
財經專家邱敏寬指出，穩茂（3105）從低檔到高檔價格可以差了3倍，這是受到疫情影響，股價也做了修正，但是PA功率放大器在未來中不可能不使用到，因此現在只不過是短期衰退，長期反而有機會再往上攻。
「複利達人怪老子」蕭世斌表示，如果以存股的角度來看，聯茂（6213）和泰碩（3338）是投資人可以留意的個股，因為它們的ROE、ROA、現金流各方面來看，獲利都是穩定的。
證券分析師丁彥鈞提到，台新金（2887）、國泰金（2882）、新光金（2888）雖然在財報、籌碼雙優股中，但是它們雖然過去一年表現好，但未來能否繼續保持實在不一定，因為降息會讓銀行利差縮小，另外如果中小企業不幸倒閉的話，銀行也會被拖累，因此從各大方面綜觀，投資金融股還是得小心留意。
----------------------------------------------------------------------

[72] 貝萊德董事總經理：2020年上半經濟萎縮11%　500萬個工作失去

▲貝萊德。（圖／路透）
實習記者曾筠淇／綜合報導
美國投資公司「貝萊德」（BlackRock）的董事總經理阿米爾.比薩特（Amer Bisat）表示，受到新冠肺炎疫情大爆發的影響，世界經濟可能會在2020年上半萎縮11%，並損失高達6萬億美元（約新台幣181億元）的經濟產值。他補充，雖然這不會像1930年代的經濟大蕭條那麼嚴重，但將會是全球第二大的經濟衝擊。
據《路透社》報導，全球最大投資公司「貝萊德」的董事總經理比薩特表示，新冠肺炎的疫情太過嚴重，世界經濟可能會在2020年上半萎縮11%，從背景上來說，這會比2008年環球金融危機的情況還要糟，也會「比人們估計的（1918年）西班牙流感期間發生的萎縮情況更慘」。
比薩特還表示，「這不會像（1930年代）全球經濟大蕭條那麼嚴重，但這肯定是我們在全球看到的第二大經濟衝擊」。
比薩特將肺炎所帶來的經濟衝濟比擬為自然災害並表示，這通常會導致事後強勁

▲5月申報所得稅時期，不少民眾擠入申報地點臨櫃報稅，增加新冠肺炎感染風險，所以國稅局現在也要開始因應與備戰。（圖／記者湯興漢攝）
記者吳靜君／台北報導
五月申報綜合所得稅，往年會湧入不少人潮臨櫃申報，部分稽徵所、國稅局甚至一日可湧入千人，人流相當可觀。在新冠肺炎（COVID-19）疫情肆虐之際，中央流行疫情指揮中心也建議，室內不要超過100人、室外也不要超過500人，且室內也要保持社交距離1.5公尺。對此，國稅局積極備戰，祭出六招抗疫、避免報稅人擠人、減少群聚感染的風險。
一，網路取號、室外搭帳棚，室內保持100人
為了因應民眾報稅，財政部賦稅署署長李慶華表示，民眾可以多利用「網路取代馬路」盡量在家裡申報完成。但是如果有民眾必須、非得要國稅局臨櫃申報，國稅局表示，除了必須量體溫、戴口罩才能入內之外，也準備好室外帳篷分散人流。國稅局表示，民眾也可多加利用線上取號，等到快輪到自己的時候再到現場即可，減少群聚機會和等候的時間。
但應該如何線上取號呢？過往只要3步驟就可完成。
●出門前透過手機或電腦到各區國稅局網站，屆時各局國稅局會建立取號及服務專區，就可以瀏覽各區國稅局、各分局所的排隊情形。
●納稅義務人可預估到達報稅地點的路程和時間，透過「線上取號」輸入手機號碼，取得QR Code條碼，民眾可擷取條碼螢幕畫面或者將條碼列印出來。
●等到了報稅地點，就可以用報稅地點的「條碼機」掃碼，就可以依序等待叫號。
國稅局官員表示，而且即使過號也沒關係，在現場呼叫3位之後可以直接插入，不用重新取號。
二，地板畫線、座位梅花座，保持社交距離
國稅局官員表示，除了要保持室內100人之外，報稅民眾間也要保持1.5公尺的社交距離。所以國稅局會在地板、地磚上畫上等待標誌，分隔等待民眾的距離。座位區方面，過往一整排的座位民眾都可以坐著等待，今年國稅局也會計算距離，部分座位不開放給民眾坐，確保等待民眾可以保持「適當的距離」。
▲為了保持社交距離，國稅局也在報稅地點畫標誌、座位區規劃限坐等，往年人擠人情況今年就不會再見。（圖／記者林敬旻攝）
三，加班時段增加、協助處理民眾報稅
每到5月申報所得稅時期，5月1日和最後一日（今年是6月1日）人流最多，國稅局及報稅地點的員工5月1日中午都會不休息加班收件；最後一日也會加班到晚上7點。但是，因應人流並且保持室內100人以下，國稅局處理民眾報稅的速度可能會下降

▲蘋果與高盛允許Apple Card用戶申請緩繳4月信用卡費，期間免收循環利息。（圖／翻攝自YouTube）
記者王曉敏／台北報導
新冠肺炎傳播為全球經濟前景蒙上陰影，為緩解用戶由於疫情引起的經濟中斷所帶來的財務壓力，蘋果與高盛（Goldman Sachs）將允許Apple Card用戶緩繳4月需繳納的信用卡費，期間免收循環利息。
據彭博報導，蘋果在致Apple Card用戶的信件中寫道：「我們知道COVID-19的情況對每個人都構成了個別的挑戰，有些客戶可能難以按月付款。」為此，蘋果與高盛將允許用戶申請緩繳4月份信用卡應付款項，對於3月份的應付款項提也提供相同的緩繳程序。
蘋果表示，若用戶曾於3月份申請緩繳，4月份的緩繳仍需再次申請。據MacRumors報導，‌Apple Card‌用戶可透過iPhone或iPad上的錢包App向Apple Card‌支援發送訊息來申請延遲付款程序，無利息的緩繳措施預計能為因受疫情影響而收入下滑的民眾提供喘息空間。
----------------------------------------------------------------------

[90] 紓困2.0／2100億特別預算比例曝光　前兩名是經濟部37％、勞動部24％

行政院長蘇貞昌2日在行政院院會後正式指派政務委員龔明鑫為「紓困陳時中」，每天必須出面向國人報告振興與紓困進度。對於紓困兩階段總規模上兆，龔明鑫表示，考量到歐美祭出有史以來最大規模振興，因此投入的1兆500億元紓困，占我國GDP5.5%至5.6％。根據2100億特別預算分配比例，又以經濟部、勞動部、交通部、衛福部這四個部會分配到最多。
新冠肺炎疫情全球持續升溫，總統蔡英文1日宣布總紓困振興方案金額將達到1兆500億元，並要求「加快加強加碼」，蘇貞昌為此也特別在2日加開臨時院會討論，並於會後率領13閣員出面說明。
蘇貞昌指出，過去三個多月來，疫情指揮中心陳時中做的很好，每天報告，讓國人同胞可以看到最快速最透明最權威最誠實的報告，做的很好，至於紓困怎麼做，這個部分也有「紓困陳時中」，就是政委龔明鑫。
蘇貞昌說，他要求龔明鑫每天出來跟國人同胞報告紓困怎麼做，也聽大家意見指教，不但防疫要做好，紓困也要做到好，振興也要做到好，關關難過關關過，希望台灣能平安。
----------------------